In [1]:
import pathlib
import re

import networkx as nx
import pandas as pd
import geopandas as gpd

In [2]:
data_dir = pathlib.Path('~/data/euris').expanduser()

In [12]:
node_paths = list(data_dir.glob('Node*'))
fairway_section_paths = list(data_dir.glob('FairwaySection*'))

In [4]:
path_re = re.compile(r'(?P<dataset>[\w]+)_(?P<country>[A-Z]{2})_(?P<date>[\d]+)\.geojson')


In [5]:
node_gdfs = []
for node_path in node_paths:
    node_gdf_i = gpd.read_file(node_path)
    match = path_re.search(str(node_path))
    if match:
        path_data = match.groupdict()
        node_gdf_i['country'] = path_data['country']
        node_gdf_i['dataset'] = path_data['dataset']
        node_gdf_i['date'] = path_data['date']
        
        
    node_gdfs.append(node_gdf_i)
node_gdf = pd.concat(node_gdfs)

In [21]:
fairway_section_gdfs = []
for fairway_section_path in fairway_section_paths:
    fairway_section_gdf_i = gpd.read_file(fairway_section_path)
    match = path_re.search(str(fairway_section_path))
    if match:
        path_data = match.groupdict()
        fairway_section_gdf_i['country'] = path_data['country']
        fairway_section_gdf_i['dataset'] = path_data['dataset']
        fairway_section_gdf_i['date'] = path_data['date']
        
        
    fairway_section_gdfs.append(fairway_section_gdf_i)
fairway_section_gdf = pd.concat(fairway_section_gdfs)
fairway_section_gdf = fairway_section_gdf.set_index('code')

/var/folders/fh/tcbrjg6n28b0lzjzh07b5t6m0000gn/T/ipykernel_5010/1427976398.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  fairway_section_gdf = pd.concat(fairway_section_gdfs)


,name,name_cb,cntrycode,cntrycode_cb,fw_code,fw_code_cb,seq_nr,seq_nr_cb,code_cb,ww_name,...,tot_length,estuary,active,ww_charges,remark,istentec,geometry,country,dataset,date
code,,,,,,,,,,,,,,,,,,,,,
HU0000100001,Duna,None,HU,None,00001,None,00001,None,None,Duna,...,46249.0,NaN,1,0.0,None,None,"LINESTRING (18.81420 45.90870, 18.81310 45.909...",HU,FairwaySection,20240527
HU0000100002,Duna,None,HU,None,00001,None,00002,None,None,Duna,...,101552.0,NaN,1,0.0,None,None,"LINESTRING (18.92280 46.17810, 18.92330 46.178...",HU,FairwaySection,20240527
HU0000100003,Duna,None,HU,None,00001,None,00003,None,None,Duna,...,7513.0,NaN,1,0.0,None,None,"LINESTRING (18.96990 47.02560, 18.97010 47.024...",HU,FairwaySection,20240527
HU0000100004,Duna,None,HU,None,00001,None,00004,None,None,Duna,...,54032.0,NaN,1,0.0,None,None,"LINESTRING (19.05090 47.43700, 19.05060 47.436...",HU,FairwaySection,20240527
HU0000100005,Duna,None,HU,None,00001,None,00005,None,None,Duna,...,286.0,NaN,1,0.0,None,None,"LINESTRING (19.05180 47.43950, 19.05150 47.438...",HU,FairwaySection,20240527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HR0000114011,Dunav,Dunav,HR,RS,00001,00001,14011,00005,RS0000114011,Dunav,...,18546.0,NaN,1,NaN,None,None,"LINESTRING (18.93079 45.54344, 18.92999 45.544...",HR,FairwaySection,20240527
HR0000113824,Dunav,Dunav,HR,RS,00001,00001,13824,00006,RS0000113824,Dunav,...,18701.0,NaN,1,NaN,None,None,"LINESTRING (19.10008 45.50149, 19.10049 45.502...",HR,FairwaySection,20240527
HR0000113635,Dunav,Dunav,HR,RS,00001,00001,13635,00007,RS0000113635,Dunav,...,69217.0,NaN,1,NaN,None,None,"LINESTRING (19.42175 45.23375, 19.42038 45.233...",HR,FairwaySection,20240527


In [7]:
node_gdf.to_file('nodes.gpkg')

In [8]:
node_gdf =  node_gdf.set_index(['country', 'objectcode'])
nodes = node_gdf.index.tolist()

In [9]:
graph = nx.DiGraph()
for n, row in node_gdf.iterrows():
    graph.add_node(n, **row)

/var/folders/fh/tcbrjg6n28b0lzjzh07b5t6m0000gn/T/ipykernel_5010/3212558283.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  row = node_gdf.loc[('FR', 'N2635')].iloc[0]


name                                                Canal du Centre
name_cb                                                        None
cntrycode                                                        FR
cntrycode_cb                                                   None
fw_code                                                       VA108
fw_code_cb                                                     None
seq_nr                                                        00002
seq_nr_cb                                                      None
code_cb                                                        None
ww_name                                               Saone - Loire
ww_name_cb                                                     None
rt_name                                             Canal du Centre
rt_name_cb                                                     None
wwauthorit                                                      VNF
wwauthorit_cb                                   

In [56]:
node = ('FR', 'N2635')
row = node_gdf.loc[node]
sectionref = row['sectionref'].iloc[0]
assert row['sectionref'].shape[0] == 2
sectionref

/var/folders/fh/tcbrjg6n28b0lzjzh07b5t6m0000gn/T/ipykernel_5010/3733193516.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  row = node_gdf.loc[node]


'FRVA10800002'

In [57]:
idx = (node_gdf['sectionref'] ==  sectionref)
source, target = node_gdf[idx].index
section = fairway_section_gdf.loc[sectionref]
section

name                                                Canal du Centre
name_cb                                                        None
cntrycode                                                        FR
cntrycode_cb                                                   None
fw_code                                                       VA108
fw_code_cb                                                     None
seq_nr                                                        00002
seq_nr_cb                                                      None
code_cb                                                        None
ww_name                                               Saone - Loire
ww_name_cb                                                     None
rt_name                                             Canal du Centre
rt_name_cb                                                     None
wwauthorit                                                      VNF
wwauthorit_cb                                   

In [60]:
graph = nx.Graph()
graph.add_edge(source, target, **section)
graph.edges[(target, source)]

{'name': 'Canal du Centre',
 'name_cb': None,
 'cntrycode': 'FR',
 'cntrycode_cb': None,
 'fw_code': 'VA108',
 'fw_code_cb': None,
 'seq_nr': '00002',
 'seq_nr_cb': None,
 'code_cb': None,
 'ww_name': 'Saone - Loire',
 'ww_name_cb': None,
 'rt_name': 'Canal du Centre',
 'rt_name_cb': None,
 'wwauthorit': 'VNF',
 'wwauthorit_cb': None,
 'cemt': 'I',
 'mdraughtcm': 180.0,
 'mlengthcm': 3850.0,
 'mlencon': 3850.0,
 'mwidthcm': 505.0,
 'mwidcon': 505.0,
 'speed': '6',
 'speedcon': '6',
 'calspeed_up': 6.0,
 'calspeed_down': 6.0,
 'calspeedc_up': 6.0,
 'calspeedc_down': 6.0,
 'maxspeed_up': 6.0,
 'maxspeed_down': 6.0,
 'maxspeedc_up': 6.0,
 'maxspeedc_down': 6.0,
 'tidedep': 0,
 'tot_length': 21115.0,
 'estuary': 0.0,
 'active': 1,
 'ww_charges': 1.0,
 'remark': '',
 'istentec': None,
 'geometry': <LINESTRING (4.646 46.863, 4.646 46.863, 4.646 46.864, 4.646 46.864, 4.646 4...>,
 'country': 'FR',
 'dataset': 'FairwaySection',
 'date': '20240527'}